In [1]:
import numpy as np
from pyflann import *
import time
import warnings

In [2]:
warnings.filterwarnings('ignore')

def sum_of_distances(points, closest, centroids):
    """
    Calculate sum of distances between the points and the centroids
    """
    dist = 0
    for j in range(points.shape[0]):
        dist += (points[j, :] - centroids[closest[j]])**2

    return np.sum(dist)


def initialize_centroids(points: np.ndarray, k: int, seed: int=1234) -> np.ndarray:
    """Returns k random centroids from the initial points
    :type k: int
    :type seed: int
    """
    c = points.copy()
    np.random.seed(seed)
    np.random.shuffle(c)
    return c[:k]

def move_centroids(points, closest, centroids, seed: int=1234):
    """
    Returns the new centroids assigned from the points closest to them.
    """
    new_centroids = []
    for k in range(centroids.shape[0]):
        # Treat the case where a centroid has no points assigned
        # by selecting a random point as a new centroid
        if len(points[closest == k]) > 0:
            new_centroids.append(points[closest == k].mean(axis=0))
        else:
            centroids = points.copy()
            np.random.seed(seed)
            idx = np.random.randint(len(points))
            new_centroids.append(points[idx])

    return np.array([points[closest == k].mean(axis=0) for k in range(centroids.shape[0])])


In [6]:
num_iterations = 30
K = 10000

# Read the SIFT data and shape them accordingly
vec = np.fromfile('data/SIFT.dat', dtype=np.uint8)
points = np.float32(vec.reshape(-1, 128))
print("Shape of data: ", points.shape)

init_centroids = initialize_centroids(points, K)

Shape of data:  (2097152, 128)


In [8]:
# K-means using randomized k-d tree for distance between points and centroids
# with 32k cluster centroids
start_time = time.time()
print("")
print("****************************************")
print("Starting approximate k-means with k-d trees (90% target precision)")
centroids = init_centroids
flann1 = FLANN()

for i in range(num_iterations):
    params = {'algorithm': 'kdtree',
              'trees': 20,
              'target_precision': 0.90}

    closest, dists = flann1.nn(centroids, points, num_neighbors=1, **params)
    centroids = move_centroids(points, closest, centroids)

print("Distance based on flann:", np.sum(dists))
distance = sum_of_distances(points, closest, centroids)
print("Total distance from assigned centroids:", distance)
print("Execution time: %s seconds: " % (time.time() - start_time))


****************************************
Starting approximate k-means with k-d trees (90% target precision)
Distance based on flann: 119097580000.0
Total distance from assigned centroids: 117545780000.0
Execution time: 14603.844402551651 seconds: 


In [10]:
# K-means using randomized k-d tree for distance between points and centroids
start_time = time.time()
print("")
print("****************************************")
print("Starting approximate k-means with k-d trees (90% target precision)")
centroids = init_centroids
flann1 = FLANN()

for i in range(30):
    params = {'algorithm': 'kdtree',
              'trees': 20,
              'target_precision': 0.90}

    closest, dists = flann1.nn(centroids, points, num_neighbors=1, **params)
    centroids = move_centroids(points, closest, centroids)
    print("Run %s, distance: %s, elapsed_time: %s" % (i, np.sum(dists), time.time()-start_time))

print("Distance based on flann:", np.sum(dists))
distance = sum_of_distances(points, closest, centroids)
print("Total distance from assigned centroids:", distance)
print("Execution time: %s seconds: " % (time.time() - start_time))


****************************************
Starting approximate k-means with k-d trees (90% target precision)
Run 0, distance: 203238620000.0, elapsed_time: 257.6802816390991
Run 1, distance: 140034920000.0, elapsed_time: 518.8287785053253
Run 2, distance: 136127800000.0, elapsed_time: 703.3943076133728
Run 3, distance: 134504595000.0, elapsed_time: 916.7052247524261
Run 4, distance: 133599850000.0, elapsed_time: 1202.4478130340576
Run 5, distance: 133035370000.0, elapsed_time: 1477.595908164978
Run 6, distance: 132545855000.0, elapsed_time: 1644.0973165035248
Run 7, distance: 132252885000.0, elapsed_time: 1890.7755753993988
Run 8, distance: 132039880000.0, elapsed_time: 2176.7464797496796
Run 9, distance: 131904225000.0, elapsed_time: 2443.4660246372223
Run 10, distance: 131710484000.0, elapsed_time: 2587.853992700577
Run 11, distance: 131559060000.0, elapsed_time: 2875.029623270035
Run 12, distance: 131429840000.0, elapsed_time: 3159.4367611408234
Run 13, distance: 131359470000.0, ela

In [7]:
start_time = time.time()
# K-means using k-means search trees for distance between points and centroids
print("")
print("****************************************")
print("Starting approximate k-means with k-means search tree (90% target precision)")
centroids = init_centroids
flann2 = FLANN()
for i in range(num_iterations):
    params = {'algorithm': 'kmeans',
            #  'branching': 100,
              'target_precision': 0.90}
    closest, dists = flann2.nn(centroids, points, num_neighbors=1, **params)
    centroids = move_centroids(points, closest, centroids)
    print("Run %s, distance: %s, elapsed_time: %s" % (i, np.sum(dists), time.time()-start_time))

print("Distance based on flann:", np.sum(dists))
print("Total distance from assigned centroids:", sum_of_distances(points, closest, centroids))
print("Execution time: %s seconds: " % (time.time() - start_time))


****************************************
Starting approximate k-means with k-means search tree (90% target precision)
Run 0, distance: 203407740000.0, elapsed_time: 131.43988466262817
Run 1, distance: 141095420000.0, elapsed_time: 261.776948928833
Run 2, distance: 137536520000.0, elapsed_time: 392.6181581020355
Run 3, distance: 136063460000.0, elapsed_time: 522.8307073116302
Run 4, distance: 135064190000.0, elapsed_time: 654.3871262073517
Run 5, distance: 134488195000.0, elapsed_time: 784.6731293201447
Run 6, distance: 134105620000.0, elapsed_time: 915.4390869140625
Run 7, distance: 133715034000.0, elapsed_time: 1045.8072052001953
Run 8, distance: 133385120000.0, elapsed_time: 1176.3599252700806
Run 9, distance: 133535050000.0, elapsed_time: 1306.9205067157745
Run 10, distance: 133303870000.0, elapsed_time: 1437.4260156154633
Run 11, distance: 133176790000.0, elapsed_time: 1567.8868687152863
Run 12, distance: 133220065000.0, elapsed_time: 1699.0954775810242
Run 13, distance: 133233020

In [8]:
num_iterations = 10 # it takes x3.5 time to compute
# K-means using exact distance
start_time = time.time()
print("****************************************")
print("Starting k-means with linear distance")
centroids = init_centroids
flann3 = FLANN()
for i in range(num_iterations):
    params = {'algorithm': 'linear'}
    closest, dists = flann3.nn(centroids, points, num_neighbors=1, **params)
    centroids = move_centroids(points, closest, centroids)
    print("Run %s, distance: %s, elapsed_time: %s" % (i, np.sum(dists), time.time()-start_time))

print("Distance based on flann:", np.sum(dists))
print("Total distance from assigned centroids:", sum_of_distances(points, closest, centroids))
print("Execution time: %s seconds: " % (time.time() - start_time))

****************************************
Starting k-means with linear distance
Run 0, distance: 189217780000.0, elapsed_time: 520.7227504253387
Run 1, distance: 134226770000.0, elapsed_time: 1024.9772317409515
Run 2, distance: 130532480000.0, elapsed_time: 1540.5842628479004
Run 3, distance: 129032710000.0, elapsed_time: 2046.3688597679138
Run 4, distance: 128192250000.0, elapsed_time: 2562.2239575386047
Run 5, distance: 127645580000.0, elapsed_time: 3074.725625038147
Run 6, distance: 127258590000.0, elapsed_time: 3581.133644580841
Run 7, distance: 126969496000.0, elapsed_time: 4088.435772418976
Run 8, distance: 126747060000.0, elapsed_time: 4607.087388038635
Run 9, distance: 126570330000.0, elapsed_time: 5114.367936372757
Distance based on flann: 126570330000.0
Total distance from assigned centroids: 126240280000.0
Execution time: 5119.254073858261 seconds: 
